In [218]:
#Imports and API Key

import pandas as pd
import quandl
from scipy import stats
import scipy
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

%aimport dataclean

quandl.ApiConfig.api_key = 'm8FYMyoCaJSbTrBASNHh'

In [219]:
#pulling data from Quandl

data = pd.read_csv('data_for_pull.csv') #staging the QUANDL keys to pull in CSV
cols = list(data['Var_name'].astype('str'))
dataset = quandl.get([val for val in data['Quandl Key']]) #looping through the QUANDL keys to pull it into one DF
dataset.columns = cols

In [220]:
#pulling FED Yield Curve data

yields = pd.read_csv('Fed10Y_3M.csv')
yields['Date'] = pd.to_datetime(yields['Date'])
yields['Date'] = yields['Date'].apply(lambda x: x.strftime('%Y-%m'))
yields = yields.set_index('Date')
yields = yields.drop(['3 Month Treasury Yield', 'Rec_prob', 'NBER_Rec','Unnamed: 7'], axis=1) 

In [221]:
## back to working on the general data
dataset.index = dataset.index.strftime('%Y-%m') #converting the datetime index to Y/M so it is collapsable
dataset = dataset.groupby(dataset.index, as_index=True).agg(sum) #collapsing by Y/M

In [222]:
#converting GDP quarterly data into monthly

dataset = dataclean.convert_q_to_m(dataset, 'GDP')

#converting consumer sentiment into monthly

dataset = dataclean.convert_q_to_m(dataset, 'CONS_SENT')

In [223]:
#calculating change in GDP and converting Y into categorical values 
dataset['Recession'] = ((dataset['GDP'] - dataset['GDP'].shift(3)) < 0).astype(int)

In [225]:
#merge fed interest rate data here
dataset = dataset.join(yields, how='outer')

In [226]:
#cutoff most of missing data, Post March 2019, Prior 1959
dataset = dataset.iloc[552:]
dataset = dataset.iloc[:-12]

In [228]:
y = dataset['Recession'] #splitting off Y

In [229]:
y_shift = y.shift(-3) #shifting y to forecast 3 months out

In [230]:
y_shift = y_shift.fillna(0)

In [231]:
dataset = dataset.drop(columns = ['GDP','Recession']) #dropping calc column and recession column from dataset, experimenting with taking out fed funds rate

In [232]:
#substituting mean value in for missing values and adding dummy column to indicate where done

for col in dataset.columns:
    dataclean.clean_zeros(col, dataset)

In [236]:
#adding momentum factors

momentum_cols = list(dataset.columns[:-11])

momentum_cols.remove('PPI') #removing PPI and CPI because they need a different transformation
momentum_cols.remove('CPI')

for i in [1,3,12]:
    for col in momentum_cols:
        dataclean.create_momentum(col,dataset,i)

In [238]:
#CPI Calcs

for i in [1,3,12]:
    for col in ['CPI','PPI']:
        dataclean.infl_momentum(col,dataset,i)

In [244]:
dataset['CPI'] - dataset['CPI'].shift(3) #there is something wrong with my CPI/PPI data

1959-01      nan
1959-02      nan
1959-03      nan
1959-04    0.000
1959-05    0.046
1959-06    0.092
1959-07    0.092
1959-08    0.092
1959-09    0.092
1959-10    0.092
1959-11    0.092
1959-12    0.046
1960-01   -0.046
1960-02    0.000
1960-03    0.000
1960-04    0.092
1960-05    0.046
1960-06    0.092
1960-07    0.046
1960-08    0.046
1960-09    0.000
1960-10    0.092
1960-11    0.092
1960-12    0.092
1961-01    0.000
1961-02    0.000
1961-03    0.000
1961-04    0.000
1961-05    0.000
1961-06    0.000
           ...  
2016-09    0.000
2016-10    0.000
2016-11    0.000
2016-12    0.000
2017-01    0.000
2017-02    0.000
2017-03    0.000
2017-04    0.000
2017-05    0.000
2017-06    0.000
2017-07    0.000
2017-08    0.000
2017-09    0.000
2017-10    0.000
2017-11    0.000
2017-12    0.000
2018-01    0.000
2018-02    0.000
2018-03    0.000
2018-04    0.000
2018-05    0.000
2018-06    0.000
2018-07    0.000
2018-08    0.000
2018-09    0.000
2018-10    0.000
2018-11    0.000
2018-12    0.0

In [243]:
dataset[['CPI','PPI','CPI_3m_shift','CPI_12m_shift']]

CPI     PPI  CPI_3m_shift  CPI_12m_shift
1959-01 13.299  31.700           nan            nan
1959-02 13.254  31.700           nan            nan
1959-03 13.254  31.700           nan            nan
1959-04 13.299  31.800         0.000            nan
1959-05 13.299  31.800         0.003            nan
1959-06 13.345  31.700         0.007            nan
1959-07 13.391  31.700         0.007            nan
1959-08 13.391  31.600         0.007            nan
1959-09 13.437  31.700         0.007            nan
1959-10 13.483  31.600         0.007            nan
1959-11 13.483  31.500         0.007            nan
1959-12 13.483  31.500         0.003            nan
1960-01 13.437  31.600        -0.003          0.010
1960-02 13.483  31.600         0.000          0.017
1960-03 13.483  31.800         0.000          0.017
1960-04 13.529  31.800         0.007          0.017
1960-05 13.529  31.700         0.003          0.017
1960-06 13.575  31.700         0.007          0.017
1960-07 13.575  31.700         0.003          0.014
1960-08 13.575  31.600         0.003          0.014
1960-09 13.575  31.600         0.000          0.010
1960-10 13.666  31.700         0.007          0.014
1960-11 13.666  31.700         0.007          0.014
1960-12 13.666  31.700         0.007          0.014
1961-01 13.666  31.800         0.000          0.017
1961-02 13.666  31.800         0.000          0.014
1961-03 13.666  31.800         0.000          0.014
1961-04 13.666  31.600         0.000          0.010
1961-05 13.666  31.500         0.000          0.010
1961-06 13.666  31.300         0.000          0.007
...        ...     ...           ...            ...
2016-09 49.970 186.900         0.000          0.000
2016-10 49.970 186.700         0.000          0.000
2016-11 49.970 186.300         0.000          0.000
2016-12 49.970 188.200         0.000          0.000
2017-01 49.970 190.700         0.000          0.000
2017-02 49.970 191.600         0.000          0.000
2017-03 49.970 191.500         0.000          0.000
2017-04 49.970 193.000         0.000          0.000
2017-05 49.970 192.800         0.000          0.000
2017-06 49.970 193.600         0.000          0.000
2017-07 49.970 193.500         0.000          0.000
2017-08 49.970 193.800         0.000          0.000
2017-09 49.970 194.800         0.000          0.000
2017-10 49.970 194.900         0.000          0.000
2017-11 49.970 195.900         0.000          0.000
2017-12 49.970 196.300         0.000          0.000
2018-01 49.970 197.900         0.000          0.000
2018-02 49.970 199.300         0.000          0.000
2018-03 49.970 199.300         0.000          0.000
2018-04 49.970 200.300         0.000          0.000
2018-05 49.970 203.200         0.000          0.000
2018-06 49.970 204.200         0.000          0.000
2018-07 49.970 204.300         0.000          0.000
2018-08 49.970 203.400         0.000          0.000
2018-09 49.970 203.600         0.000          0.000
2018-10 49.970 204.600         0.000          0.000
2018-11 49.970 201.500         0.000          0.000
2018-12 49.970 202.100         0.000          0.000
2019-01 49.970 198.600         0.000          0.000
2019-02 49.970 198.700         0.000          0.000

[722 rows x 4 columns]

In [ ]:
### Data Prep Finished Here ###